In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np

/Users/shehvincent/anaconda3/envs/edge_computing/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Set GPU device
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

IndexError: list index out of range

In [202]:
import pandas as pd
import glob

path = r'./Dataset/MachineLearningCVE' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, encoding='cp1252', index_col=None, header=0)
    li.append(df)
    print("Read Completed for ", filename)

df = pd.concat(li, axis=0, ignore_index=True)

df = df.rename(columns={' Label': 'Label'})

Read Completed for  ./Dataset/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Read Completed for  ./Dataset/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv
Read Completed for  ./Dataset/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv
Read Completed for  ./Dataset/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Read Completed for  ./Dataset/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Read Completed for  ./Dataset/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv
Read Completed for  ./Dataset/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv
Read Completed for  ./Dataset/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv


In [203]:
df["Label"].value_counts()

Label
BENIGN                          2273097
DoS Hulk                         231073
PortScan                         158930
DDoS                             128027
DoS GoldenEye                     10293
FTP-Patator                        7938
SSH-Patator                        5897
DoS slowloris                      5796
DoS Slowhttptest                   5499
Bot                                1966
Web Attack ï¿½ Brute Force         1507
Web Attack ï¿½ XSS                  652
Infiltration                         36
Web Attack ï¿½ Sql Injection         21
Heartbleed                           11
Name: count, dtype: int64

In [204]:
print(df.shape)

(2830743, 79)


In [205]:
from scipy.stats import zscore

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

In [206]:
del sorted_ds


NameError: name 'sorted_ds' is not defined

In [208]:

#drop na values and reset index
data_clean = df.dropna().reset_index(drop=True)

# Checkng for DUPLICATE values
data_clean.drop_duplicates(keep='first', inplace = True)

data_clean['Label'].value_counts()

print("Read {} rows.".format(len(data_clean)))

Read 2522009 rows.


In [209]:
analyze(df)


2830743 rows
**  Destination Port:53805 (1%)
**  Flow Duration:1050899 (37%)
**  Total Fwd Packets:1432 (0%)
**  Total Backward Packets:1747 (0%)
** Total Length of Fwd Packets:17928 (0%)
**  Total Length of Bwd Packets:64698 (2%)
**  Fwd Packet Length Max:5279 (0%)
**  Fwd Packet Length Min:384 (0%)
**  Fwd Packet Length Mean:99716 (3%)
**  Fwd Packet Length Std:253909 (8%)
** Bwd Packet Length Max:4838 (0%)
**  Bwd Packet Length Min:583 (0%)
**  Bwd Packet Length Mean:147614 (5%)
**  Bwd Packet Length Std:248869 (8%)
** Flow Bytes/s:1593909 (56%)
**  Flow Packets/s:1240164 (43%)
**  Flow IAT Mean:1166311 (41%)
**  Flow IAT Std:1056642 (37%)
**  Flow IAT Max:580289 (20%)
**  Flow IAT Min:136316 (4%)
** Fwd IAT Total:493098 (17%)
**  Fwd IAT Mean:737737 (26%)
**  Fwd IAT Std:700313 (24%)
**  Fwd IAT Max:437316 (15%)
**  Fwd IAT Min:110631 (3%)
** Bwd IAT Total:414928 (14%)
**  Bwd IAT Mean:670824 (23%)
**  Bwd IAT Std:709042 (25%)
**  Bwd IAT Max:368285 (13%)
**  Bwd IAT Min:66074 (2%

In [210]:
# data_clean=data_clean.drop('Flow ID',axis=1)
# data_clean=data_clean.drop(' Source IP',axis=1)
# data_clean=data_clean.drop(' Destination IP',axis=1)
# data_clean=data_clean.drop(' Timestamp',axis=1)

In [211]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder



#drop na values and reset index
data_clean = data_clean.dropna().reset_index(drop=True)

# label encoding
labelencoder = LabelEncoder()
data_clean['Label'] = data_clean["Label"].map({'BENIGN':0}).fillna(1).astype(int)
data_clean['Label'] = labelencoder.fit_transform(data_clean['Label'])

data_clean['Label'].value_counts()

Label
0    2096134
1     425875
Name: count, dtype: int64

In [212]:
data_clean.shape

(2522009, 79)

In [213]:
data_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
data_clean.dropna(inplace=True)

benign_data = data_clean.loc[data_clean["Label"] == 0].copy()
benign_data = benign_data.sample(frac=1, random_state=4022).reset_index(drop=True)




mid_idx = benign_data.shape[0]//2
test_benign = benign_data.iloc[mid_idx:].copy()
benign_data = benign_data.iloc[:mid_idx] #Data to train the AE

malicious_data = data_clean.loc[data_clean["Label"] != 0].copy()
mixed_data = pd.concat([test_benign, malicious_data])
mixed_data.shape

(1473270, 79)

In [214]:
benign_train_X = benign_data.drop(columns=["Label"], axis=1)
benign_train_Y = benign_data["Label"].values
mixed_X = benign_data.drop(columns=["Label"], axis=1)
mixed_Y = benign_data["Label"].values

#Test Benign and Malicious
benign_test_X = test_benign.drop(columns=["Label"], axis=1)
benign_test_Y = test_benign["Label"].values

mal_test_X = malicious_data.drop(columns=["Label"], axis=1)
mal_test_Y = malicious_data["Label"].values



In [215]:
mal_test_X

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
64539,444,42664070,5,6,141,291,93,0,28.200000,38.861292,...,4,20,0.0,0.0000,0,0,0.0,0.000,0,0
66518,444,2030,2,0,12,0,6,6,6.000000,0.000000,...,1,20,0.0,0.0000,0,0,0.0,0.000,0,0
70523,444,119991834,5523,5525,2866110,33406,1274,0,518.940793,204.203530,...,5522,20,997842.0,831471.3076,1585781,409903,8143987.5,1821773.648,9432176,6855799
71004,444,5005212,15,15,0,90,0,0,0.000000,0.000000,...,0,28,0.0,0.0000,0,0,0.0,0.000,0,0
71543,444,119973135,4376,4374,2321478,26244,1273,6,530.502285,262.768815,...,4375,20,0.0,0.0000,0,0,0.0,0.000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452879,80,5005388,5,3,599,2021,599,0,119.800000,267.880944,...,1,32,0.0,0.0000,0,0,0.0,0.000,0,0
2452959,80,5016759,5,5,599,4149,599,0,119.800000,267.880944,...,1,32,0.0,0.0000,0,0,0.0,0.000,0,0
2453199,80,5009163,4,4,599,2021,599,0,149.750000,299.500000,...,1,32,0.0,0.0000,0,0,0.0,0.000,0,0
2453208,80,73,1,1,0,0,0,0,0.000000,0.000000,...,0,32,0.0,0.0000,0,0,0.0,0.000,0,0


In [216]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(benign_train_X)
X_mixed = scaler.transform(mixed_X)
X_ben_test_scaled = scaler.transform(benign_test_X)
X_mal_test_scaled = scaler.transform(mal_test_X)

# Split the data set into training and testing
# X_train, X_test, Y_train, Y_test = train_test_split(
#     X_scaled, Y, test_size=0.25, random_state=2, shuffle=True)


print('X.shape = ',X_train_scaled.shape)
print('Y.shape = ',X_train_scaled.shape)

print('X.shape = ',X_mixed.shape)
print('Y.shape = ',X_mixed.shape)

X.shape =  (1047528, 78)
Y.shape =  (1047528, 78)
X.shape =  (1047528, 78)
Y.shape =  (1047528, 78)


In [217]:
from keras import regularizers
from keras.layers import Dense, LeakyReLU, BatchNormalization, Input, LSTM, TimeDistributed, RepeatVector, Lambda
from keras import Input
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras import regularizers

In [218]:
# reshape inputs for LSTM [samples, timesteps, features]
X_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
print("Training data shape:", X_scaled.shape)


Training data shape: (1047528, 1, 78)


# LSTM-AE Model

In [152]:
# define the autoencoder network model
def autoencoder_model(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(196, activation='relu', return_sequences=True, 
              kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(125, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = LSTM(125, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(196, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    return model

# With learning rate=0.0001 and mse loss function.

In [153]:
# create the autoencoder model
model = autoencoder_model(X_scaled)

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=opt, loss='mse', metrics=['accuracy']) 
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1, 78)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 1, 196)         │       215,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 125)            │       161,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_3 (RepeatVector)  │ (None, 1, 125)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 1, 125)         │       125,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ (None, 1, 196)         │       252,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 1, 78)          │        15,366 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,914 (2.94 MB)

 Trainable params: 769,914 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [221]:
# fit the model to the data
history = model.fit(X_scaled, X_scaled, epochs=30, batch_size=128, validation_split=0.05).history

Epoch 1/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 66s 9ms/step - accuracy: 0.8970 - loss: 0.0287 - val_accuracy: 0.9181 - val_loss: 0.0044
Epoch 2/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 64s 8ms/step - accuracy: 0.9182 - loss: 0.0342 - val_accuracy: 0.9130 - val_loss: 0.0034
Epoch 3/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 65s 8ms/step - accuracy: 0.9227 - loss: 0.0129 - val_accuracy: 0.8938 - val_loss: 0.0030
Epoch 4/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 64s 8ms/step - accuracy: 0.9294 - loss: 0.0152 - val_accuracy: 0.9332 - val_loss: 0.0030
Epoch 5/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 71s 9ms/step - accuracy: 0.9332 - loss: 0.0176 - val_accuracy: 0.9206 - val_loss: 0.0020
Epoch 6/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 67s 9ms/step - accuracy: 0.9346 - loss: 0.0234 - val_accuracy: 0.9405 - val_loss: 0.0022
Epoch 7/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 66s 8ms/step - accuracy: 0.9317 - loss: 0.0139 - val_accuracy: 0.9468 - val_loss: 0.0019
Epoch 8/30
4240/7775 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.9336 - loss: 0

KeyboardInterrupt: 

In [156]:
import time

# Measure inference time
start_time = time.time()
pred = model.predict(X_scaled)
end_time = time.time()

inference_time = end_time - start_time
print("Inference time: {:.2f} seconds".format(inference_time))


39388/39388 ━━━━━━━━━━━━━━━━━━━━ 36s 916us/step
Inference time: 41.72 seconds


## Plot the distribution

In [220]:
model.evaluate(X_scaled,X_scaled, batch_size=128)

8184/8184 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.8024 - loss: 0.0462


[0.043775513768196106, 0.8023375272750854]

In [175]:
X_ben_test_scaled = X_ben_test_scaled.reshape(X_ben_test_scaled.shape[0], 1, X_ben_test_scaled.shape[1])
model.evaluate(X_ben_test_scaled,X_ben_test_scaled,batch_size=128)

9847/9847 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9435 - loss: 0.2041


[0.14310511946678162, 0.9433718919754028]

In [219]:
X_mal_test_scaled = X_mal_test_scaled.reshape(X_mal_test_scaled.shape[0], 1, X_mal_test_scaled.shape[1])
model.evaluate(X_mal_test_scaled,X_mal_test_scaled,batch_size=128)

3327/3327 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8023 - loss: 0.1875


[0.31289592385292053, 0.6158486008644104]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def get_losses(X):
  predictions = model.predict(X)

  # Calculate losses
  mse_loss = mean_squared_error()
  losses = mse_loss(X_scaled, predictions).numpy()

  # If you need to calculate loss for each data point individually:
  individual_losses = np.mean(np.square(X_scaled - predictions), axis=1)
  return individual_losses

def plot_distributions(individual_losses):
  # Set the style of seaborn for better aesthetics
  sns.set_theme(style="whitegrid")

  # Plotting the distribution of losses
  plt.figure(figsize=(10, 6))
  sns.histplot(individual_losses, kde=True, color="blue", binwidth=0.01)
  plt.title('Distribution of Losses')
  plt.xlabel('Loss')
  plt.ylabel('Frequency')
  plt.show()

In [ ]:
train_losses = get_losses(X_scaled)
plot_distributions(train_losses)

In [ ]:
ben_test_losses = get_losses(X_ben_test_scaled)
plot_distributions(ben_test_losses)

In [ ]:
mal_test_losses = get_losses(X_mal_test_scaled)
plot_distributions(mal_test_losses)

# With default learning rate and mae loss function, to 30 epochs.

In [224]:
# create the autoencoder model
model = autoencoder_model(X_scaled)

model.compile(optimizer='adam', loss='mae', metrics=['accuracy']) 
model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 1, 78)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_20 (LSTM)                  │ (None, 1, 196)         │       215,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_21 (LSTM)                  │ (None, 125)            │       161,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_5 (RepeatVector)  │ (None, 1, 125)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_22 (LSTM)                  │ (None, 1, 125)         │       125,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_23 (LSTM)                  │ (None, 1, 196)         │       252,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 1, 78)          │        15,366 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,914 (2.94 MB)

 Trainable params: 769,914 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [225]:
# fit the model to the data
history = model.fit(X_scaled, X_scaled, epochs=30, batch_size=128, validation_split=0.05).history

Epoch 1/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 66s 8ms/step - accuracy: 0.6114 - loss: 0.0723 - val_accuracy: 0.8725 - val_loss: 0.0183
Epoch 2/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 68s 9ms/step - accuracy: 0.9080 - loss: 0.0155 - val_accuracy: 0.9116 - val_loss: 0.0112
Epoch 3/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 65s 8ms/step - accuracy: 0.9278 - loss: 0.0113 - val_accuracy: 0.9156 - val_loss: 0.0099
Epoch 4/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 69s 9ms/step - accuracy: 0.9328 - loss: 0.0098 - val_accuracy: 0.9553 - val_loss: 0.0082
Epoch 5/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 64s 8ms/step - accuracy: 0.9406 - loss: 0.0083 - val_accuracy: 0.9249 - val_loss: 0.0069
Epoch 6/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 57s 7ms/step - accuracy: 0.9437 - loss: 0.0073 - val_accuracy: 0.9518 - val_loss: 0.0064
Epoch 7/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 61s 8ms/step - accuracy: 0.9464 - loss: 0.0070 - val_accuracy: 0.9471 - val_loss: 0.0060
Epoch 8/30
7775/7775 ━━━━━━━━━━━━━━━━━━━━ 62s 8ms/step - accuracy: 0.9472 - loss: 0

In [41]:
import time

# Measure inference time
start_time = time.time()
pred = model.predict(X_train_scaled)
end_time = time.time()

inference_time = end_time - start_time
print("Inference time: {:.2f} seconds".format(inference_time))

pred = np.argmax(pred,axis=1)
# y_test = Y_test.argmax(axis=1)

22093/22093 [==============================] - 48s 2ms/step
Inference time: 53.66 seconds


In [42]:
import datetime as dt
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, auc
from sklearn import preprocessing


start = dt.datetime.now()

y_pred = model.predict(X_test)
escore = model.evaluate(X_test, X_test, batch_size=128)

# evaluate classification metrics (for multiclass targets)
#accuracy = accuracy_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1))
#precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
#recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
#f1 = f1_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
#confusion = confusion_matrix(Y_test.argmax(axis=1), y_pred.argmax(axis=1))



# print the results
#print("Accuracy: {:.3f}".format(accuracy))
#print("Precision: {:.3f}".format(precision))
#print("Recall: {:.3f}".format(recall))
#print("F1-score: {:.3f}".format(f1))
#print("Confusion Matrix:\n", confusion)

print('Completed')
print('Time taken:',dt.datetime.now()-start)

5524/5524 [==============================] - 14s 3ms/step - loss: 0.0005 - accuracy: 0.9999
Completed
Time taken: 0:01:09.123427
